In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import mmread
from scipy.sparse.linalg import cg, spilu, LinearOperator, aslinearoperator, spsolve_triangular
from scipy.sparse import isspmatrix, csr_matrix, tril, diags

In [3]:
# Function to create matrix A
def generate_matrix_A(n):
    diagonals = [
        -1 * np.ones(n - 1),  # Lower diagonal
        2 * np.ones(n),       # Main diagonal
        -1 * np.ones(n - 1)   # Upper diagonal
    ]
    
    # Create the sparse matrix
    A = diags(diagonals, offsets=[-1, 0, 1], format='csr')
    
    # Adjust the last diagonal element
    A[-1, -1] = 1  # Update the last diagonal element directly in the sparse matrix
    
    return A

In [4]:
# Function to create vector b
def generate_vector_b(n):
    b = np.zeros(n)
    b[0] = 1
    return b

In [5]:
def ichol_sparse_optimized(A): #final version
    """
    Optimized Incomplete Cholesky factorization for a sparse matrix A.
    A should be a symmetric positive definite matrix in CSR format.
    """
    A = A.tocsr()  # Ensure A is in CSR format
    n = A.shape[0]
    L = A.copy()   # Create a copy of A to store the result

    for k in range(n):
        # Update the diagonal element
        L[k, k] = np.sqrt(L[k, k])
        
        # Update column k below the diagonal
        rows = L.indices[L.indptr[k] : L.indptr[k + 1]]
        rows = rows[rows > k]  # Only consider rows below the diagonal
        
        if rows.size > 0:
            L[rows, k] /= L[k, k]

        # Update the lower-right submatrix
        for i in rows:
            L[i, rows] -= L[i, k] * L[rows, k]
    
    # Extract the lower triangular part of L
    L = tril(L, format='csr')
    return L


In [6]:
def conjugate_gradient_with_preconditioning(A, b, x0=None, tol=1e-6, max_iter=1000, verbose=False):
    if x0 is None:
        x0 = np.zeros_like(b)
    x = x0.copy()
    
    # Compute incomplete Cholesky factorization (Preconditioner)
    L = ichol_sparse_optimized(A)
    
    # Initial residual
    r = b - A @ x
    
    # Apply preconditioner: Solve L y = r
    y = spsolve_triangular(L, r, lower=True)
    # Then solve L^T z = y
    z = spsolve_triangular(L.T, y, lower=False)
    
    p = z.copy()
    rs_old = np.dot(r.T, z)
    
    if np.sqrt(rs_old) < tol:  # Initial guess is the solution
        if verbose:
            print("Initial guess is the solution.")
        return x, [np.sqrt(rs_old)], [x], 0
    
    residuals = [np.sqrt(np.dot(r.T, r))]
    solutions = [x.copy()]
    
    for i in range(max_iter):
        Ap = A @ p
        alpha = rs_old / np.dot(p.T, Ap)
        x += alpha * p
        r -= alpha * Ap
        
        # Apply preconditioner to the new residual
        y = spsolve_triangular(L, r, lower=True)
        z = spsolve_triangular(L.T, y, lower=False)
        
        rs_new = np.dot(r.T, z)
        
        residuals.append(np.sqrt(np.dot(r.T, r)))
        solutions.append(x.copy())
        
        if np.sqrt(np.dot(r.T, r)) < tol:
            if verbose:
                print(f"Converged in {i + 1} iterations with residual norm {np.sqrt(np.dot(r.T, r)):.2e}.")
            break
        
        beta = rs_new / rs_old
        p = z + beta * p
        rs_old = rs_new
    else:
        if verbose:
            print(f"Did not converge within {max_iter} iterations. Final residual norm: {np.sqrt(np.dot(r.T, r)):.2e}")
    
    return x, residuals, solutions, i + 1

In [7]:
n=10000
A = generate_matrix_A(n)
b = generate_vector_b(n)
# Assuming A is a symmetric positive definite sparse matrix and b is the right-hand side vector
#x0 = np.zeros_like(b)
#x0 = np.ones(n)
x0 = np.random.rand(n)
# x0_random = np.random.rand(n)
# x0 = np.ones(n) - x0_random
solution, residuals, solutions, iterations = conjugate_gradient_with_preconditioning(A, b, x0=x0, verbose=True)


Converged in 1 iterations with residual norm 1.34e-14.


In [8]:
def preconditioner_spilu(A):
    """
    Compute the incomplete LU decomposition with zero fill-in (ILU0).
    """
    ilu = spilu(A)
    Mx = lambda x: ilu.solve(x)
    return Mx

In [9]:
def conjugate_gradient_with_preconditioning_spilu(A, b, x0=None, tol=1e-6, max_iter=1000, verbose=False):
    if x0 is None:
        x0 = np.zeros_like(b)
    x = x0.copy()

    # Compute preconditioner using ILU
    Mx = preconditioner_spilu(A)

    # Initial residual
    r = b - A @ x

    # Apply preconditioner
    z = Mx(r)

    p = z.copy()
    rs_old = np.dot(r.T, z)

    residuals = [np.sqrt(np.dot(r.T, r))]
    solutions = [x.copy()]

    for i in range(max_iter):
        Ap = A @ p
        denominator = np.dot(p.T, Ap)
        if np.abs(denominator) < 1e-12 or np.isnan(denominator):
            if verbose:
                print("Numerical instability detected in denominator. Terminating early.")
            break
        alpha = rs_old / denominator
        if np.isnan(alpha) or np.isinf(alpha):
            if verbose:
                print("Invalid alpha encountered. Terminating early.")
            break
        x += alpha * p
        r -= alpha * Ap

        # Apply preconditioner to the new residual
        z = Mx(r)

        rs_new = np.dot(r.T, z)

        residual_norm = np.sqrt(np.dot(r.T, r))
        residuals.append(residual_norm)
        solutions.append(x.copy())

        if residual_norm < tol:
            if verbose:
                print(f"Converged in {i + 1} iterations with residual norm {residual_norm:.2e}.")
            break

        beta = rs_new / rs_old
        if np.isnan(beta) or np.isinf(beta):
            if verbose:
                print("Invalid beta encountered. Terminating early.")
            break
        p = z + beta * p
        rs_old = rs_new
    else:
        if verbose:
            print(f"Did not converge within {max_iter} iterations. Final residual norm: {residual_norm:.2e}")

    return x, residuals, solutions, i + 1


In [10]:
n=10000
h = 0.01
A = generate_matrix_A(n)
b = generate_vector_b(n)
# Assuming A is a symmetric positive definite sparse matrix and b is the right-hand side vector
#x0 = np.zeros_like(b)
#x0 = np.ones(n)
#x0 = np.random.rand(n)
x0_random = np.random.rand(n)
x0 = np.ones(n) - x0_random
solution, residuals, solutions, iterations = conjugate_gradient_with_preconditioning(A, b, x0, verbose=True)


Converged in 1 iterations with residual norm 1.34e-14.
